In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import pygsheets
import json

load_dotenv()

True

In [2]:
service_account = pygsheets.authorize(service_account_file='JSONs\\spreadsheet-automator-449222-28b7c907226f.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1p3ecWHthVRXvAEr900K8YOEt_c2ey98v8ym3I_GTWEA/edit?usp=sharing')


In [3]:
test = sheet.worksheet('title', 'Test')

In [4]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests

API_KEY= os.environ.get('RIOT_API_KEY')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):

    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+API_KEY)
        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+API_KEY)
        return response.json()['puuid']

def get_idtag_from_puuid(puuid=None):
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key'+API_KEY)

    id = {
        'gameName' : response.json()['gameName'],
        'tagLine' : response.json()['tagLine']
    }

    return id

In [5]:
def get_ladder(top=None,):
    root = 'https://na1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    masters = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'
    chall_response = requests.get(root + chall + '?api_key=' + API_KEY)
    gm_response = requests.get(root + gm + '?api_key=' + API_KEY)
    masters_response = requests.get(root + masters + '?api_key=' + API_KEY)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, masters_df]).reset_index(drop=True)[0:top]
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] = ladder['rank'] + 1
    
    return ladder

def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?queue=450&type=normal&start={start}&count={count}'

    response = requests.get(root_url + endpoint + query_params +'&api_key=' + API_KEY)


    return response.json()

def get_match_data_from_id(region=None, matchId=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{matchId}'

    response = requests.get(root_url+endpoint+'?api_key='+API_KEY)

    return response.json()

In [6]:
get_ladder(300)

KeyError: 'entries'

In [10]:
get_puuid(gameName="JustSayWhen", tagLine = "NA1")

'qyBucbVWn1FU8GSXtx1_X0br-ac8Fy-_KdQ7_1F7pbZr2WGdX15Xg4vNIJs-3-AH9otHfRMu2B-pmQ'

In [8]:
get_match_history(region="americas", puuid="5kXhUQSJDvpYKmmsKN5OzSvDhel0y2YxGrgCMsl_CabC_8NXPZTCJTi5KYWSxa62sIrPEVHaQ8FVyw")

['NA1_5244564564',
 'NA1_5244560754',
 'NA1_5244104494',
 'NA1_5230697102',
 'NA1_5230691836',
 'NA1_5230683804',
 'NA1_5230672335',
 'NA1_5230660632',
 'NA1_5230644837',
 'NA1_5230632194',
 'NA1_5230617957',
 'NA1_5229371512',
 'NA1_5228182838',
 'NA1_5221819613',
 'NA1_5216854883',
 'NA1_5216832915',
 'NA1_5215873552',
 'NA1_5215859385',
 'NA1_5215844362',
 'NA1_5215824821']

In [9]:
get_match_data_from_id(region="americas", matchId="NA1_5216832915")

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_5216832915',
  'participants': ['mneoTO3yZX_chscFBDtaq5DUa71iv7JPpm4b3JZNdXlatnU06i5DeeaL--V7yQnJIZMzTCgXCYQnpA',
   '-XFGWuLGP2XRPJgifetzoBx8iyuIbLpGpBpUtQmdU_RdaSPAOn_JvzJzsSQHlfLVzkOPEiLvG9MMMw',
   '0HlQfdzTPItB_V4uGxNruhC7Yv6SXqZNWJorayy5id1pIySAEatXCVsPBLr6oT9rSuOuQd4VuIDSCA',
   'rABKhti1kYDfsZLI0pngeO1o8K0GHwE01PbQ0pI9v6KMUBt9UkgqlgJ7NREcI0jCqyQVuVQrqRveCA',
   'uyDVAXf4U4umLMPeW-ev8QiBXSu1W_khHsMdZ0CVzsqkHJgUkVfqpQBQlRhgZcKwlDmTHGvNKr9Mlg',
   '2jHfLQ8MGeLTdz2HOu-Cco8y4VTOuHm1L1gUmf0RmJpp0p93EzlylsicU_9eQsyCzWqmJgRtjFbMEg',
   '5kXhUQSJDvpYKmmsKN5OzSvDhel0y2YxGrgCMsl_CabC_8NXPZTCJTi5KYWSxa62sIrPEVHaQ8FVyw',
   'XQbgZrSaGzaT8DU7DNIi_HxqyA0C2bO2kvLEyPWU_MyzEtZFxUjoA11xZX8_piBeTKfgtlBimeOESA',
   'l8BZ1u2siw9H7suEmWmZEQgUhpfItIVRDsVaL5Fj8dpKWxLP7SrTsAzL7IpLDvxuINNzun-7ld0UKw',
   'j3Xku5paUGAL7RWIk--pGRjZlqW29W6sKIRI1nLMeyKKUud8i32mw6aiZ0tdIkHoegQqDouNpZ-BSw']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1738

In [ ]:
#match scraper
matches = get_match_history(region="americas", puuid='qyBucbVWn1FU8GSXtx1_X0br-ac8Fy-_KdQ7_1F7pbZr2WGdX15Xg4vNIJs-3-AH9otHfRMu2B-pmQ', count=100)
with open('snowballs.json', 'r') as file:
    snowballs = json.load(file)
friendsList = {'denial#viego', '23ejd#NA1', 'NightIsShadow#NA1', 'demon slayer yi#NA1', 'Alias Mudd#NA1', 'Azirs Pantaloons#Azir', 'Kiramdzz#1204', 'OriKitty#meow'}
flag = False
count = 0
for x in matches:
    if flag:
        break
    match_data = get_match_data_from_id(region = 'americas', matchId = x)
    for player in match_data["info"]["participants"]:
        puuid = player['puuid']
        index = -1
        for i, p in enumerate(snowballs):
            if p["puuid"] == puuid:
                index = i
                break
        snowballsHit=0
        name = player['riotIdGameName']
        tagline = player['riotIdTagline']
        profileIconId = 0
        url = f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={API_KEY}'
        profile = (requests.get(url))
        if profile.status_code == 200:
            profileIconId = (profile.json())['profileIconId']
        else:
            print(profile.status_code)
            flag = True
            break
        url = f'https://na1.api.riotgames.com/lol/challenges/v1/player-data/{puuid}?api_key={API_KEY}'
        challengeData = (requests.get(url))
        if challengeData.status_code == 200:
            challengeData = (challengeData.json())['challenges']
            challenge_id_to_find = 101203
            value = None
            for j, challenge in enumerate(challengeData):
                if challenge["challengeId"] == challenge_id_to_find:
                    value = challenge["value"]
                    break
            if value is not None:
                    snowballsHit = value
            else:
                print(challengeData.status_code)
                flag = True
                break
            data = {
                    "puuid" : puuid,
                    "name" : name,
                    "tagline" : tagline,
                    "snowballsHit" : snowballsHit,
                    "profileIconId" : profileIconId
                    }
            if (index == -1):
                snowballs.append(data)
            else:
                snowballs[index] = data
with open('snowballs.json', 'w') as file:
    json.dump(snowballs, file)
print(snowballs)


429
[{'puuid': 'qyBucbVWn1FU8GSXtx1_X0br-ac8Fy-_KdQ7_1F7pbZr2WGdX15Xg4vNIJs-3-AH9otHfRMu2B-pmQ', 'name': 'JustSayWhen', 'tagline': 'NA1', 'snowballsHit': 225188.0, 'profileIconId': 6400}, {'puuid': 'vcsWJvj3TLLZGzY-O5T-whqWRBxwG9Ycq9y3y2adpMNBVP8Z7K5DXDlaqkD261lF5m8JfH4SYPNtQg', 'name': 'MeowSick', 'tagline': 'NA1', 'snowballsHit': 72118.0, 'profileIconId': 5414}, {'puuid': 'NUBDbN_AJ2-O6NSrcr5p0hR5AoqUVSGOLxe9jB8cbTxzGFsGsLLhAAsL3DEmqGWPaX3QDz0sRmfrbg', 'name': 'NightShiftGnomey', 'tagline': 'NA1', 'snowballsHit': 40669.0, 'profileIconId': 1296}, {'puuid': 'EyI0pXJJ-iX6mMb3WrNqSk1AOI2-deEDn7r7BjRMAsANlsV9RQDDx1qE3lfJLfSDsN4T6jwgPjFD_g', 'name': 'StandardSumo', 'tagline': 'NA1', 'snowballsHit': 24811.0, 'profileIconId': 5621}, {'puuid': 'kNkYyIiphkhKLWIKIAsj6KuxHUhSwt9cWsBPt9nCj5W07Wmm32-TNeCrxri5l2sdupQmOrCVpaKN8g', 'name': 'CNO', 'tagline': '650', 'snowballsHit': 22067.0, 'profileIconId': 3457}, {'puuid': 'QdeQCwtqcp_SvCb7OpbKP58geDgsBc64g61M2Jbr6w9lavNvUf7rk6onsV3d1zm8WMJSa86Sk60lMg

In [12]:
#sort function
with open('snowballs.json', 'r') as file:
    snowballs = json.load(file)
snowballs = sorted(snowballs,  key=lambda x: x['snowballsHit'], reverse=True)
with open('snowballs.json', 'w') as file:
    json.dump(snowballs, file)